In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import albumentations as A
import albumentations.pytorch
import torchvision
import numpy as np
from PIL import Image

In [ ]:
decoder = torchvision.transforms.ToPILImage()
transforms = A.Compose([
    A.pytorch.ToTensorV2()
])

In [ ]:
origin = Image.open('xichun.jpg').convert("RGB").resize((512, 512))
plt.imshow(origin)
plt.show()
nimage = np.array(origin)

fimage = np.array(origin, dtype=np.float32)
plt.imshow(fimage.astype(np.uint8))
plt.show()

In [ ]:
x = torch.FloatTensor(fimage).permute(2, 1, 0).unsqueeze(0)
print(x.shape)
t = x.squeeze(0)
print(t.shape)

output = t.detach().permute(2, 1, 0).numpy().astype(np.uint8)
plt.imshow(output)
plt.show()

In [ ]:
seed = 6666
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

conv = nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1)
output = conv(x)

print(output.shape)

output = output.squeeze(0)
plt.imshow(output.detach().permute(2, 1, 0).numpy().astype(np.uint8) / 255.0)
plt.show()
